In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import math

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class C_L(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=256, layer_dim=8):
        super(C_L, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.input_dim = input_dim

        # stacked LSTM
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.layer_dim, dropout=0.4, bidirectional=False, batch_first=True)

    def forward(self, x, s_h, s_c):
        # Initialize hidden and cell state
        h0 = s_h
        c0 = s_c
        out, (hn, cn) = self.lstm(x, (h0,c0))
        return out, hn, cn


class C_F(nn.Module):
    def __init__(self, hidden_dim=64, output_dim=24):
        super(C_F, self).__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.bn1 = nn.BatchNorm1d(num_features=self.hidden_dim)
        self.L_out1 = nn.Linear(self.hidden_dim, self.output_dim) 
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, id=-1):
        linear_out = self.L_out1(self.bn1(x[:,id,:]))
        linear_out = self.sigmoid(linear_out)
        return linear_out

# prepare train and val set
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
class RnnDataset(TensorDataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    
    def __getitem__(self, index):
        return self.data[index],self.label[index] 
    
    def __len__(self):
        return len(self.data)

def read_dataset(sensor_id=4004,start_point='1966-12-18 14:30:00',train_point='2014-01-20 9:30:00'):

    if(str(sensor_id)[0]=='4'):
        trainX = pd.read_csv('./data'+'/reservoir_stor_'+str(sensor_id)+'_sof24.tsv', sep='\t')
    elif(str(sensor_id)[0]=='6'):
        trainX = pd.read_csv('./data'+'/raingauge_byhour_'+str(sensor_id)+'_sof.tsv', sep='\t')
    else:
        print("Error: the support sensor type is 4*** or 6****.")
        sys.exit()
        
    start_num = trainX[trainX["TSC_TSTAMP_UTC"]==start_point].index.values[0]
    print("for sensor ", sensor_id, "start_num is: ", start_num)
    idx_num = 0
    print(len(trainX))
    print(trainX[:3])

    val_skips = []
    
    val_set = pd.read_csv('./data/'+str(sensor_id)+'_validation_timestamps_24avg.tsv', sep='\t')
    val_points = val_set["Hold Out Start"]
    for val_i in val_points:
        valskip = trainX[trainX["TSC_TSTAMP_UTC"]==val_i].index.values[0] - start_num
        val_skips.append(valskip)
    
    # id of test_start and test_end, save it in the last two values in val_skips
    test_start = trainX[trainX["TSC_TSTAMP_UTC"]=='2017-01-01 14:30:00'].index.values[0] - start_num 
    test_end = trainX[trainX["TSC_TSTAMP_UTC"]=='2018-12-31 14:30:00'].index.values[0] - start_num
    val_skips.append(test_start)
    val_skips.append(test_end)   
        
    print("val_skips is: ", val_skips)

    #foot label of train_end
    train_end = trainX[trainX["TSC_TSTAMP_UTC"]==train_point].index.values[0] - start_num 
    print("train_end is : ", train_end)

    #the whole dataset
    train_x = trainX[start_num:] 
    print(len(train_x))
    #train_end = len(train_x) # the whole data set? It's your choice
    sensor_data = np.array(train_x["TSC_VALUE_F"])
    time_data = [math.floor(int(np.array(train_x["TSC_TSTAMP_UTC"])[i][5:7])-1) for i in range(len(train_x))]
    
    return sensor_data, train_end, val_skips, time_data, start_num
 
def diff_norm_dataset(sensor_data0):
    a = sensor_data0
    b = a[:-1]
    a = a[1:] - b
    c = np.array([0]+a.tolist())
    mean = c.mean()
    print("mean is: ", mean)
    std = c.std()
    print("std is ", std)
    c = (c - mean) / std

    extream_num = 0
    print("extream boundary is: ", std * alpha_extreams)
    total_num = len(c)
    for i in range(total_num):
        if(abs(c[i]) > alpha_extreams):
            extream_num += 1
    print(" the total norm data number is: ", total_num)
    print(" the number of extreams is: ", extream_num, " the ratio is: ", extream_num/total_num)

    return c, mean, std

def std_norm_dataset(sensor_data0):

    c = sensor_data0
    mean = c.mean()
    print("mean is: ", mean)
    std = c.std()
    print("std is ", std)
    c = (c - mean) / std
  
    return c, mean, std


def split_dataset(norm_data,train_end, input_dim=24, output_size=24, train_days=90, predict_days=3, train_volum=50000):

    en_seq_len = train_days
    de_seq_len = predict_days

    DATA = []
    Label = []
    print("norm data size:", len(norm_data))

    train_size = train_end + 1 - (train_days * input_dim + output_size * predict_days)
    print("Available train size is: ", train_size)
    
    # step 1, randomly choose train data from normed data
    norm_num = train_volum * norm_percent
    extreme_num = train_volum * (1 - norm_percent)
    extreme_total = 0
    norm_total = 0
    
    # avoid points in validation set and whole test set years
    val_skips_extend = []
    l = len(val_skips)
    for i in range(l-2):
        for j in range(output_size*predict_days):
            val_skips_extend.append(val_skips[i]+j)
    
    test_start = val_skips[l-2]
    print("test set start ID is: ", test_start)
    test_end = val_skips[l-1]
    print("test set end ID is: ", test_end)
    
    while(extreme_total<extreme_num or norm_total<norm_num):
        i = random.randint(0, train_size)
    
        if (i+train_days not in val_skips_extend ) and ((i+train_days)<test_start or (i+train_days)>test_end): 
            data0 = np.array(norm_data[i:(i+train_days*input_dim)]).reshape(train_days,-1)
            label0 = np.array(class_label[(i+train_days*input_dim):(i+train_days*input_dim+output_size*predict_days)]) 
            label1 = np.array(norm_data[(i+train_days*input_dim):(i+train_days*input_dim+output_size*predict_days)]) 

            # step 2, over sampling extreme points
            if(label0.sum()>=1 and extreme_total<extreme_num):
                extreme_total += 1;
                DATA.append(data0)
                Label.append(label0)

            if(label0.sum()<1 and norm_total<norm_num):
                norm_total += 1;
                DATA.append(data0)
                Label.append(label0) 
    
    dataset1 = RnnDataset(DATA, Label)
    data_loader = DataLoader(dataset1, 
                         batch_size,
                         shuffle=True,
                         num_workers=2,
                         pin_memory=True,
                         collate_fn=lambda x: x)
    return data_loader

#initiate models
def create_model(input_dim, hidden_dim, layer_dim, output_dim):
    encoder = C_L(input_dim, hidden_dim, layer_dim)
    outlinear = C_F(hidden_dim, predict_days) 

    encoder = encoder.to(device)
    outlinear = outlinear.to(device)

    criterion = nn.BCELoss(reduction='sum')
    criterion1 = nn.MSELoss(reduction='sum')
    encoder_optimizer = torch.optim.SGD(encoder.parameters(),0.001)  
    outlinear_optimizer = torch.optim.Adam(outlinear.parameters(),0.0005) 
    return encoder, outlinear, encoder_optimizer, outlinear_optimizer, criterion, criterion1

def train_loop(max_accuracy):
    num_epochs = 100 #early stop or reach to num_epochs
    early_stop = 0
    old_val_acc = 0
    for epoch in range(num_epochs):
        print_loss_total = 0  # Reset every epoch
        encoder.train()
        outlinear.train()
        for i, batch in enumerate(dataloader):
            x_train = [TrainData for TrainData, _ in batch]
            y_train = [TrainLabel for _, TrainLabel in batch]
            x_train = torch.from_numpy(np.array(x_train, np.float32)).to(device)
            y_train = torch.from_numpy(np.array(y_train,np.float32)).to(device)

            # Clear gradients w.r.t. parameters
            encoder_optimizer.zero_grad()
            outlinear_optimizer.zero_grad()
            loss = 0

            # Forward pass
            s_h = torch.zeros(layer_dim, x_train.size(0), hidden_dim).to(device)
            s_c = torch.zeros(layer_dim, x_train.size(0), hidden_dim).to(device)
            encoder_out, _, _ = encoder(x_train, s_h, s_c)
            e_out = outlinear(encoder_out)
        
            # C loss parameters  alpha=2  beta=0.5
            loss = 0.5 * criterion(e_out**2, y_train) + 0.5 * criterion1(e_out, y_train)
        
            loss.backward()
            encoder_optimizer.step()
            outlinear_optimizer.step()
            print_loss_total += loss.item()

        encoder.eval()
        outlinear.eval()
        val_acc, max_accuracy = generate_val_accuracy(max_accuracy, epoch)
        print('-----------Epoch: {}. train_Loss>: {:.6f}. --------------------'.format(epoch, print_loss_total)) 
        print('-----------Epoch: {}. val_f1_score>: {:.6f}. --------------------'.format(epoch, val_acc)) 

        if val_acc == 1:
            break

        #early stop
        if(val_acc < old_val_acc):
            early_stop += 1
        else:
            early_stop = 0
        if(early_stop >= 4):     # C early stop is 4
            break
        old_val_acc = val_acc
    return max_accuracy

def inference_test(encoder, outlinear, x_test):
    
    encoder.eval()
    outlinear.eval()
    y_predict = []
    with torch.no_grad():
        x_test = torch.from_numpy(np.array(x_test, np.float32)).to(device)
        s_h = torch.zeros(layer_dim, x_test.size(0), hidden_dim).to(device)
        s_c = torch.zeros(layer_dim, x_test.size(0), hidden_dim).to(device)
        encoder_out, encoder_h, encoder_c = encoder(x_test, s_h, s_c)
        d_out = outlinear(encoder_out)
        y_predict.extend(d_out[0])
        y_predict = [y_predict[i].item() for i in range(len(y_predict))]
        y_predict = np.array(y_predict).reshape(1,-1) 
        
    return y_predict

def test(test_point):
    
    encoder.eval()
    outlinear.eval()
    test_predict = np.zeros(predict_days*output_dim)
    #test_data
    if(str(sensor_id)[0]=='4'):
        trainX = pd.read_csv('./data'+'/reservoir_stor_'+str(sensor_id)+'_sof24.tsv', sep='\t')
    elif (str(sensor_id)[0]=='6'):
        trainX = pd.read_csv('./data'+'/raingauge_byhour_'+str(sensor_id)+'_sof.tsv', sep='\t')
    else:
        print("Error: the support sensor type is 4*** or 6****.")
        sys.exit()
        
    #foot label of test_data
    point = trainX[trainX["TSC_TSTAMP_UTC"]==test_point].index.values[0]
    test_point = point - start_num
    x = trainX[point-train_days*input_dim:point]["TSC_TSTAMP_UTC"]
    y = class_label[test_point:test_point+predict_days*output_dim]

    #inference
    norm_data = sensor_data_norm
    x_test = np.array(norm_data[test_point-train_days*input_dim:test_point], np.float32).reshape(train_days, -1)
    x_test = [x_test]
    y_predict = inference_test(encoder=encoder,outlinear=outlinear,x_test=x_test)
    y_predict = np.array(y_predict.tolist())[0]
    
    return y_predict,y

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
def generate_val_accuracy(max_accuracy, epoch):
    encoder.eval()
    outlinear.eval()
    test_predict = np.zeros(predict_days*output_dim)
    #test_data
    val_set = pd.read_csv('./data/'+str(sensor_id)+'_validation_timestamps_24avg.tsv',sep='\t')
    val_points = val_set["Hold Out Start"]

    total = 0
    val_loss_list = []
    val_pred_list = []
    val_pred_lists_print = []
    ground_truth_lists = []
    for i in range(len(val_points)):
        val_pred_list_print = []
        ground_truth_list = []
        val_point = val_points[i]
        test_predict, ground_truth = test(test_point=val_point)
        test_predict = [1 if test_predict[i].item()>=0.5 else 0 for i in range(len(test_predict))]  
    
        for j in range(len(test_predict)):
            val_pred_list_print.append(test_predict[j])
            ground_truth_list.append(ground_truth[j])
    
        val_pred_lists_print.append(val_pred_list_print)
        ground_truth_lists.append(ground_truth_list)
    
    val_acc = f1_score(np.array(ground_truth_lists).reshape(1,-1).squeeze(), np.array(val_pred_lists_print).reshape(1,-1).squeeze(), average='micro')
    new_max_accuracy = max_accuracy
                  
    if (val_acc > max_accuracy) :
        aa = pd.DataFrame(data=val_pred_lists_print)
        aa.to_csv('./val/'+str(sensor_id)+norm+'classification.tsv', sep='\t')
        #save_model
        torch.save(outlinear, outlinear_name) 
        torch.save(encoder, encoder_name)
        new_max_accuracy = val_acc
    print("val max F1 score: ", new_max_accuracy)

    return val_acc, new_max_accuracy

def generate_test(path, filename):
    test_predict = np.zeros(predict_days*output_dim)
    #test_data
    val_set = pd.read_csv('./data/'+str(sensor_id)+'_test_timestamps_24avg.tsv',sep='\t')
    val_points = val_set["Start"]
    total = 0
    val_pred_list = []
    val_pred_lists_print = []
    ground_truth_lists = []
    
    for i in range(len(val_points)):
        val_pred_list_print = []
        ground_truth_list = []
        val_point = val_points[i]
        test_predict, ground_truth = test(test_point=val_point)
        test_predict = [1 if test_predict[i].item() >= 0.5 else 0 for i in range(len(test_predict))]  
    
        for j in range(len(test_predict)):
            val_pred_list_print.append(test_predict[j])
            ground_truth_list.append(ground_truth[j])
            
        val_pred_lists_print.append(val_pred_list_print)
        ground_truth_lists.append(ground_truth_list)
                  
    aa = pd.DataFrame(data = val_pred_lists_print)
    aa.to_csv(path+filename+'.tsv', sep='\t')
    print("test classification is saved at:  " + path + filename + '.tsv')


In [ ]:
from matplotlib import pyplot as plt

sensor_ids = [4009]

for i in range(len(sensor_ids)) :
    # sensor's data
    sensor_id = sensor_ids[i]
    
    # dataset parameters
    start_point = '1980-12-31 14:30:00'
    max_accuracy = 0
    print("Begin!! Sensor id is :---------------------",sensor_id)

    # training parameters
    input_dim = 1          # univari forecasting, set 1
    output_dim = 1           # univari forecasting, set 1
    train_days = 15*24       # history vector length, h=360
    train_end = 0
    predict_days = 24*3      # forecasting vector length, f=72
    train_point = '2019-12-31 14:30:00'
    alpha_extreams = 1.5     # normal/extreme boundary, \epsilon=1.5
    print("alpha_extreams is: ", alpha_extreams)
    norm_percent = 0.0       # C use oversampling, OS%=1
    is_diff = 1              # one order difference, set 1
    if(is_diff==0):
        norm = "std"
    else:
        norm = "diff"

    val_skips = []
    sensor_data, train_end, val_skips, time_data, start_num = read_dataset(sensor_id=sensor_id,start_point=start_point,train_point=train_point)

    if(is_diff==0):
        sensor_data_norm, mean, std = std_norm_dataset(sensor_data)
        print("I am using std_norm.")
    else:
        sensor_data_norm, mean, std = diff_norm_dataset(sensor_data)
        print("I am using diff_std_norm.")
    
    class_label = []
    for i in range(len(sensor_data_norm)):
        if abs(sensor_data_norm[i]) > alpha_extreams :
            class_label.append(1)
        else:
            class_label.append(0)

    # model hyperparameters
    hidden_dim = 1024         # C hidden = 1024
    layer_dim = 4             # C layers = 4
    batch_size = 64           # C batch size = 8
    train_volum = 1000      # C volume = 100000
    print("hidden dim is: ", hidden_dim)
    print("layer dim is: ", layer_dim)
    print("train_volum is: ", train_volum)

    encoder_name = "./model/" + str(sensor_id) + '_' + norm + "LSTM_classifier.txt"  # C_LSTM model path
    outlinear_name = "./model/" + str(sensor_id) + '_' + norm + "FC_classifier.txt"  # C_FC model path

    #get train data_loader 
    dataloader = split_dataset(norm_data=sensor_data_norm, train_end=train_end, input_dim=input_dim, output_size=output_dim, train_days=train_days, predict_days=predict_days,train_volum=train_volum)

    #create_model
    encoder, outlinear, encoder_optimizer, outlinear_optimizer,criterion,criterion1 = create_model(input_dim=input_dim, hidden_dim=hidden_dim, layer_dim=layer_dim, output_dim=input_dim)

    #train the model
    train_loop(max_accuracy)    #if only inferencing, just hide this line.
    
    print("Finish!! Sensor id is :---------------------",sensor_id)
    
    #generate test prediction    
    encoder = torch.load(encoder_name)
    outlinear = torch.load(outlinear_name)
    encoder.eval()
    outlinear.eval()
    generate_test("./test/", str(sensor_id) + 'classification')  # the path of generated test classification
